In [1]:
%pwd

'/Users/jvh/Desktop/Porsche Challenge/hpi-porsche-challenge'

In [24]:
import numpy as np
from darts import TimeSeries
import pandas as pd

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
import darts.utils.timeseries_generation as tg
from darts.datasets import AirPassengersDataset, EnergyDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihood
from darts.models import NaiveSeasonal
from darts.metrics import rmse
from darts.models import *

import warnings

warnings.filterwarnings("ignore")
import logging

import matplotlib.pyplot as plt

In [3]:
data = pd.read_parquet("time-series-model/data/processed/energy_data_processed.parquet")
data.set_index("timestamp", inplace=True)

test_split = 0.2
test_split_index = int(len(data) * (1 - test_split))

train, test = data[:test_split_index], data[test_split_index:]
train, test = TimeSeries.from_dataframe(train, fill_missing_dates=True), TimeSeries.from_dataframe(test, fill_missing_dates=True)

del data

In [33]:
def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(test))
    test_rmse = rmse(test, forecast)
    print(f"model {model} obtains RMSE: {test_rmse:.2f}")

# Multivariate models
eval_model(NaiveSeasonal(K=1))
eval_model(NaiveSeasonal(K=7))
eval_model(NaiveSeasonal(K=24))
eval_model(NaiveSeasonal(K=168))
eval_model(VARIMA())
eval_model(XGBModel())
eval_model(NBEATSModel())
eval_model(RNNModel(model="LSTM", input_chunk_length=24*7, training_length=24*7*4, output_chunk_length=24, hidden_dim=32, n_rnn_layers=2, n_epochs=100, random_state=42))

model NaiveSeasonal() obtains RMSE: 3301.44
model NaiveSeasonal(K=7) obtains RMSE: 4029.18
model NaiveSeasonal(K=24) obtains RMSE: 3314.10
model NaiveSeasonal(K=168) obtains RMSE: 3488.57
